In [5]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.manifold import TSNE
import xgboost as xgb
from hyperopt import fmin, tpe, space_eval, hp, STATUS_OK, Trials
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras import Sequential
from keras.regularizers import l1, l2
from numpy.random import default_rng
import matplotlib.lines as mlines

%matplotlib inline

In [59]:
df = pd.read_csv('2020.1 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.csv', skiprows=9)
df = df[df['Salario mensual BRUTO (en tu moneda local)'] < 1_000_000]
df = df[df['Años en la empresa actual'] < 40]
df = df[(df['Salario mensual BRUTO (en tu moneda local)'] >= 10_000) & (df['Salario mensual BRUTO (en tu moneda local)'] <= 1_000_000)]
df = df.reset_index().drop('index', axis=1)
df.head()

,Me identifico,Tengo,Dónde estás trabajando,Años de experiencia,Años en la empresa actual,Años en el puesto actual,¿Gente a cargo?,Nivel de estudios alcanzado,Estado,Carrera,...,¿Tenés algún tipo de discapacidad?,¿Sentís que esto te dificultó el conseguir trabajo?,¿A qué eventos de tecnología asististe en el último año?,Cantidad de empleados,Actividad principal,¿La recomendás como un buen lugar para trabajar?,¿Cómo calificás las políticas de diversidad e inclusión?,¿A cuántos kilómetros de tu casa queda la oficina?,Beneficios extra,"¿Cuáles considerás que son las mejores empresas de IT para trabajar en este momento, en tu ciudad?"
0,Hombre,40,Catamarca,15.0,0.0,0.0,0,Universitario,Completado,Licenciatura en Sistemas de Información,...,NaN,NaN,NaN,201-500,Otras industrias,6,6,4.0,"Comidas pagas / subvencionadas, Estacionamient...",NaN
1,Mujer,35,Catamarca,15.0,10.0,3.0,5,Posgrado,Completado,Licenciatura en Sistemas de Información,...,NaN,No,NaN,11-50,Servicios / Consultoría de Software / Digital,5,8,2.0,Abono de celular y/o Internet,NaN
2,Hombre,39,Chaco,10.0,5.0,5.0,6,Universitario,Incompleto,Ingeniería en Sistemas de Información,...,NaN,NaN,Ekoparty,11-50,Servicios / Consultoría de Software / Digital,10,10,1.0,"Capacitaciones y/o cursos, Clases de idiomas, ...",NaN
3,Hombre,42,Chaco,18.0,3.0,3.0,0,Universitario,Completado,Licenciatura en Análisis de Sistemas,...,NaN,No,NaN,11-50,Producto basado en Software,5,3,-1.0,"Horarios flexibles, Sueldo dolarizado, WFH (Tr...",NaN
4,Hombre,29,Chaco,4.0,0.0,0.0,1,Universitario,Completado,Ingeniería Electrónica,...,NaN,NaN,"Pynea, Autodefensa digital",201-500,Otras industrias,8,9,5.0,"Abono de celular y/o Internet, Comidas pagas /...",NaN


In [61]:
def clean_row(row):
    d = {}
    for k, v in row.items():
        for word in clean_words(v):
            d[f'{k}={word}'] = 1
    return pd.Series(d, dtype='int')
    
def clean_words(value):
    value = value.replace('Microsoft Azure (Tables, CosmosDB, SQL, etc)', 'Microsoft Azure(TablesCosmosDBSQLetc)')
    value = value.replace('Snacks, golosinas, bebidas', 'snacks')
    value = value.replace('Descuentos varios (Clarín 365, Club La Nación, etc)', 'descuentos varios')
    value = value.replace('Sí, de forma particular', 'de forma particular')
    value = value.replace('Sí, los pagó un empleador', 'los pagó un empleador')
    value = value.replace('Sí, activa', 'activa')
    value = value.replace('Sí, pasiva', 'pasiva')
    return [clean_word(v) for v in value.split(',') if clean_word(v)]

def clean_word(word):
    val = str(word).lower().strip().replace(".", "")
    if val in ('ninguno', 'ninguna', 'no', '0', 'etc)', 'nan'):
        return ''
    if val == 'Microsoft Azure(TablesCosmosDBSQLetc)':
        return 'Microsoft Azure (Tables, CosmosDB, SQL, etc)'
    if val in ('pycon', 'pyconar'):
        return 'pyconar'
    if val in ('nodeconf', 'nodeconfar'):
        return 'nodeconfar'
    if val in ('meetup', 'meetups'):
        return 'meetups'
    if val == 'snacks':
        return 'snacks, golosinas, bebidas'
    if val == 'descuentos varios':
        return 'descuentos varios (clarín 365, club la nación, etc)'
    return val.replace(' ', '')

enc = OneHotEncoder(handle_unknown='ignore')
df_encoded = pd.concat((
pd.DataFrame(
enc.fit_transform(
        df[[
            'Me identifico',
            'Nivel de estudios alcanzado',
            'Universidad',
            'Estado',
            'Carrera',
            '¿Contribuís a proyectos open source?',
            '¿Programás como hobbie?',
            'Trabajo de',
            '¿Qué SO usás en tu laptop/PC para trabajar?',
            '¿Y en tu celular?',
            'Tipo de contrato',
            'Orientación sexual',
            'Cantidad de empleados',
            'Actividad principal',
        ]].fillna('')
    ).toarray()
)
,
df[[
    '¿Tenés guardias?',
    'Realizaste cursos de especialización',
    '¿A qué eventos de tecnología asististe en el último año?',
    'Beneficios extra',
    'Plataformas',
    'Lenguajes de programación',
    'Frameworks, herramientas y librerías',
    'Bases de datos',
    'QA / Testing',
    'IDEs',
    'Lenguajes de programación'
]].fillna('').apply(clean_row, axis=1).fillna(0),
df[['¿Gente a cargo?', 'Años de experiencia', 'Tengo', 'Salario mensual BRUTO (en tu moneda local)']],
), axis=1)
df_encoded

,0,1,2,3,4,5,6,7,8,9,...,¿A qué eventos de tecnología asististe en el último año?=zruput,¿A qué eventos de tecnología asististe en el último año?=ágiles,¿A qué eventos de tecnología asististe en el último año?=ágileslatam,¿A qué eventos de tecnología asististe en el último año?=ágileslatinoamerica,¿Tenés guardias?=activa,¿Tenés guardias?=pasiva,¿Gente a cargo?,Años de experiencia,Tengo,Salario mensual BRUTO (en tu moneda local)
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,15.0,40,41000.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5,15.0,35,60000.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,10.0,39,180000.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,18.0,42,150000.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,4.0,29,25200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5761,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,9.0,40,252450.0
5762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,33,76000.0
5763,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,4.0,26,39900.0
5764,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,11.0,32,43000.0


In [62]:
X_train, X_test, y_train, y_test = train_test_split(df_encoded.drop('Salario mensual BRUTO (en tu moneda local)', axis=1), df['Salario mensual BRUTO (en tu moneda local)'], test_size=0.2, random_state=42)
X_train

,0,1,2,3,4,5,6,7,8,9,...,¿A qué eventos de tecnología asististe en el último año?=yc,¿A qué eventos de tecnología asististe en el último año?=zruput,¿A qué eventos de tecnología asististe en el último año?=ágiles,¿A qué eventos de tecnología asististe en el último año?=ágileslatam,¿A qué eventos de tecnología asististe en el último año?=ágileslatinoamerica,¿Tenés guardias?=activa,¿Tenés guardias?=pasiva,¿Gente a cargo?,Años de experiencia,Tengo
5233,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,20.0,46
4807,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,30
2057,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,14.0,33
3167,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.0,27
2759,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,8.0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,29
5191,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7.0,40
5226,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,5.0,33
5390,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,7.0,25


In [73]:
model = Sequential(
    [
        Input(shape=(X_train.shape[1],)),
        Dense(1000, activation="relu", name="layer1"),
        Dense(1000, activation="relu", name="layer2"),
        Dense(1000, activation="relu", name="layer3"),
        Dense(1000, activation="relu", name="layer4"),
        Dense(1, name="output"),
    ]
)
model(np.asarray(X_train, dtype=np.float), np.asarray(y_train, dtype=np.float));
model.compile(loss='mean_squared_error', optimizer='Adam')
model.fit(
    np.asarray(X_train, dtype=np.float),
    np.asarray(y_train, dtype=np.float),
    batch_size=64,
    epochs=200,
);
r2_score(y_test, model.predict(np.asarray(X_test, dtype=np.float)))

Epoch 1/200
73/73 [==============================] - 5s 64ms/step - loss: 7108246528.0000
Epoch 2/200
73/73 [==============================] - 5s 63ms/step - loss: 3706085888.0000
Epoch 3/200
73/73 [==============================] - 5s 64ms/step - loss: 3138294272.0000
Epoch 4/200
73/73 [==============================] - 5s 69ms/step - loss: 2811519744.0000
Epoch 5/200
73/73 [==============================] - 5s 73ms/step - loss: 2610762496.0000 0s - loss:
Epoch 6/200
73/73 [==============================] - 5s 66ms/step - loss: 2255499520.0000
Epoch 7/200
73/73 [==============================] - 5s 65ms/step - loss: 2027814528.0000
Epoch 8/200
73/73 [==============================] - 5s 73ms/step - loss: 1888448384.0000
Epoch 9/200
73/73 [==============================] - 5s 68ms/step - loss: 1830519040.0000
Epoch 10/200
73/73 [==============================] - 5s 71ms/step - loss: 1836295552.0000
Epoch 11/200
73/73 [==============================] - 6s 76ms/step - loss: 1620232832.00

73/73 [==============================] - 5s 63ms/step - loss: 111167160.0000
Epoch 93/200
73/73 [==============================] - 5s 64ms/step - loss: 57284132.0000
Epoch 94/200
73/73 [==============================] - 5s 62ms/step - loss: 30805724.0000
Epoch 95/200
73/73 [==============================] - 5s 67ms/step - loss: 37012244.0000
Epoch 96/200
73/73 [==============================] - 5s 68ms/step - loss: 41683900.0000
Epoch 97/200
73/73 [==============================] - 5s 69ms/step - loss: 41956136.0000
Epoch 98/200
73/73 [==============================] - 6s 77ms/step - loss: 21445044.0000
Epoch 99/200
73/73 [==============================] - 6s 80ms/step - loss: 20440870.0000
Epoch 100/200
73/73 [==============================] - 7s 90ms/step - loss: 20562132.0000
Epoch 101/200
73/73 [==============================] - 5s 67ms/step - loss: 36928312.0000
Epoch 102/200
73/73 [==============================] - 6s 76ms/step - loss: 39892232.0000
Epoch 103/200
73/73 [=========

73/73 [==============================] - 4s 60ms/step - loss: 15181071.0000
Epoch 184/200
73/73 [==============================] - 4s 61ms/step - loss: 13576545.0000
Epoch 185/200
73/73 [==============================] - 5s 63ms/step - loss: 25806902.0000
Epoch 186/200
73/73 [==============================] - 5s 62ms/step - loss: 21033028.0000
Epoch 187/200
73/73 [==============================] - 5s 63ms/step - loss: 15052017.0000
Epoch 188/200
73/73 [==============================] - 5s 62ms/step - loss: 11296026.0000
Epoch 189/200
73/73 [==============================] - 5s 62ms/step - loss: 9125709.0000
Epoch 190/200
73/73 [==============================] - 4s 61ms/step - loss: 13138533.0000
Epoch 191/200
73/73 [==============================] - 5s 62ms/step - loss: 28289588.0000
Epoch 192/200
73/73 [==============================] - 5s 65ms/step - loss: 77886928.0000
Epoch 193/200
73/73 [==============================] - 5s 64ms/step - loss: 62423416.0000
Epoch 194/200
73/73 [====

0.4274417111518676